<a href="https://colab.research.google.com/github/LukeClancy/MachineLearning_DecisionTree/blob/master/Copy_of_ML_H1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from math import log
from google.colab import files
import time

# Calculate the Gini index for a subset of the dataset
#def gini_index(groups, classes):
#   # count all samples at split point
#   num_instances = float(sum([len(group) for group in groups]))

#   gini = 0.0 # sum weighted Gini index for each group
#   for group in groups:
#      size = float(len(group))
#      if size == 0: # avoid divide by zero
#         continue
#      score = 0.0
#      # score the group based on the score for each class
#      for class_val in classes:
#         p = [row[-1] for row in group].count(class_val) / size
#         score += p * p
      # weight the group score by its relative size
#      gini += (1.0 - score) * (size / num_instances)
#   return gini

#credit where credit due: https://stackoverflow.com/questions/889900/accurate-timing-of-functions-in-python
def timeWrapper(method):
  def wrap(*args, **kw):
    strt = int(time.time() * 1000)
    result = method(*args, **kw)
    endTime = int(time.time() * 1000)
    print("time was: " + str(endTime - strt))
    return result
  return wrap
    

def info_gain(groups, classes):
  num_instances = float(sum([len(group) for group in groups]))
  infGain = float(0)
  for group in groups:
    size = float(len(group))
    if size == 0:
      continue
    #for row in group:
    #  posies += row[-1]
    #negies = float(size - posies)    
    entropy = float(0)
    for classy in classes: #in this case only two
      count = [row[-1] for row in group].count(classy)
      if count != 0:
        entropy -= (count/size) * log(count/size, 2)
    infGain += (size / num_instances) * (1 - entropy)
  return infGain
  
# Create child splits for a node or make a leaf node
def split(node, max_depth, depth):
   print('in split len groups: ' + str(len(node['groups'])))
   print('len grp 0: ' + str(len(node['groups'][0])) + ", grp 1: " + str(len(node['groups'][1])))
   left, right = node['groups']
   del(node['groups'])
   # check for a no split
   if not left or not right:
      print('in check for no split')
      node['left'] = node['right'] = create_leaf(left + right)
      return
   # check for max depth
   if depth >= max_depth:
      print('in check for max depth')
      node['left'], node['right'] = create_leaf(left), create_leaf(right)
      return
   print('selecting left')
   node['left'] = select_attribute(left)
   split(node['left'], max_depth, depth+1)
   print('selecting right')
   node['right'] = select_attribute(right)
   split(node['right'], max_depth, depth+1)


# split the dataset based on an attribute and attribute value
def test_split(index, value, dataset):
   left, right = list(), list()
   for row in dataset:
      if row[index] < value:
         left.append(row)
      else:
         right.append(row)
   return left, right


# Select the best split point for a dataset
@timeWrapper
def select_attribute(dataset):
   print("len selecting: " + str(len(dataset)))
   class_values = list(set(row[-1] for row in dataset))
   b_index, b_value, b_score, b_groups = 999, 999, 0, None
   for index in range(len(dataset[0])-1):
      for row in dataset:
         groups = test_split(index, row[index], dataset)
         infg = info_gain(groups, class_values)
         if infg > b_score: #changed < to >
            b_index, b_value, b_score, b_groups = index, row[index], infg, groups
      #if index % 100 == 0:
        #print("index " + str(index) + " completed")
   return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a leaf node class value
def create_leaf(group):
   outcomes = [row[-1] for row in group]
   return max(set(outcomes), key=outcomes.count)

# Build a decision tree
def build_tree(train, max_depth):
   print("len train: " + str(len(train)))
   root = select_attribute(train)
   print("attr selected")
   split(root, max_depth, 1)
   return root
  
  
# Print a decision tree
def print_tree(node, depth=0):
   if depth == 0:
      print 'Tree:'
   if isinstance(node, dict):
      print('%s[X%d < %.3f]' % (depth*' ', (float(node['index'])+1), float(node['value'])))
      print_tree(node['left'], depth+1)
      print_tree(node['right'], depth+1)
   else:
      print('%s[%s]' % ((depth*' ', node)))
       
   # Make a prediction with a decision tree
def predict(node, row):
   if row[node['index']] < node['value']:
      if isinstance(node['left'], dict):
         return predict(node['left'], row)
      else:
         return node['left']
   else:
      if isinstance(node['right'], dict):
         return predict(node['right'], row)
      else:
         return node['right']

        
if __name__ == "__main__":
   print('available files:')
   !ls
   _files = files.upload()
   print('files entered ' + str(_files.keys()))
   fileName = input("please enter filename chosen, it doesnt like dots so use \"s ")
   myFile = open(fileName, "r")
   line = myFile.readline()
   print('past a, line:' + line)
   dataset = []
   while line and line != "":
      dat = line.split(',')
      for a in range(len(dat) - 1):
            dat[a] = float(dat[a])
      dataset.append(dat)
      line = myFile.readline()
   print('past b')
   test_dat = dataset[0:len(dataset)/3]
   train_dat = dataset[len(dataset)/3:]
   tree = build_tree(train_dat, 4)
   print_tree(tree)
   correct = 0
   failed = 0
   for row in test_dat:
      prediction = predict(tree, row)
      if prediction == row[-1]:
        correct+=1
      else:
        failed+=1
      #print('Predicted=%d, Ground truth=%d' % (prediction, row[-1]))
   print('correct - failed ratio: %d - %d' % (correct, failed))

available files:
har.csv  sample_data


files entered []
please enter filename chosen, it doesnt like dots so use "s "har.csv"
past a, line:1.4450396e-001,1.8926326e-001,6.2769317e-002,-9.0429967e-001,-1.8193654e-001,-4.4315051e-001,-9.0110019e-001,-1.1081268e-001,-4.0059935e-001,-9.3189578e-001,4.2098681e-002,-3.3652580e-001,7.1647623e-001,6.5581306e-001,8.0898559e-001,-5.3697289e-001,-9.9147358e-001,-7.9322292e-001,-8.1428623e-001,-8.9712813e-001,-1.5862471e-001,-4.0196407e-001,-4.8472227e-001,5.5583606e-001,1.7380770e-001,-4.1077609e-002,1.3076571e-002,7.3864994e-002,-3.1871724e-001,-2.7818848e-001,-1.0415121e-001,4.6032895e-001,-1.1399824e-001,-1.9921508e-001,-9.8337991e-002,5.2228156e-002,2.6252715e-001,-9.9084317e-001,-9.5033550e-001,9.8594537e-001,8.7409866e-001,4.2479149e-002,3.3309449e-001,-8.1418938e-001,-2.5461649e-001,-3.4244329e-001,-8.0702901e-001,-2.4906181e-001,-3.5495765e-001,8.6053314e-001,1.5158093e-001,4.0688316e-001,8.4957326e-001,-1.9754314e-001,1.1301998e-001,8.1578730e-002,6.7244755e-001,-9.5460358e-0


Luke Clancy
Machine Learning
Jan 18, 2019

                                 Assignment #1

1: I could not figure out how to paste an image in here, so this is the link 

https://drive.google.com/file/d/1IebFkhDMY5cO5K8F0QkztSvR-M4bCThppA/view?usp=sharing

  1 - height/width ration of the 'o'
  
  2 - height/width ratio of the 'g'
  
  3 - tail of the 'u' apparent
  
  4 - tail of the 'g' passes the stem
  
  5 - angle at the end of 'g''s tail
  
  6 - overall smoothness vs rigidity of the lines
  
  7 - space and consistency of space between characters
  
  8 - bite size of the 'u' compared to the size of the character as a whole
  
  9 - string angling in a certain direction smoothly and consistently 
  
  10 - consistency of the characters likenesses to eachother in general.
  
  ________________________________________________________________________

2: (left for false right for true)

  1 - 

                    A
                  f/ \t
                  B  False
                f/ \t
             True   False

  2 - 
  
                     C
                   f/ \t
                   A   True
                 f/  \t
               False   B
                     f/ \t
                 False   True

 3 - 
 
                      A
                   f/    \t
                   B      B
                f/ \t    f/ \t
            False True  True False
            
________________________________________________________________________

3:

gain(F1) = 4/6

gain(F2) = 0

gain(F3) = 0

We would choose F1 as it has the highest information gain. Notice that you can branch off the A option (what is stopping information gain from equalling one) with either F2 or F3 to complete the tree with an overall information gain of 1.

notes, please disregard:

   entropy = -(x1/xtotal * log2 (x1/xtotal) + x2/xtotal * log2(x1/xtotal) + ... )

   information gain = 1 - xtotal /count (entropy(x)) + ytotal/count (entropy(y))...
   
_____________________________________________________________________________

4:

We can determine the extent of overfit by comparing the Accuracy peak to the accuracy(node count) we are comparing it to.

Oa(x) = A(n) - A(x), where n is the nodecount where efficiency peaks (in this case 15)

I woud choose nodecount of 15 a it is halfway between the two peaks, 10 and 20.

_____________________________________________________________________________

5:

The performance is definitely better than a random guess, but it seems that it only needed one feature
to reliably decide whether someone is standing or sitting. aka the tree only needed to be of depth 1. The feature that was used was the 84th feature.

This was with a ratio of 1166 correct to 0 incorrect on the test data after being trained on the training data. 

I find this unusual although it may be a good lesson on the importance of features.

______________________________________________________________________________


# 

**Homework Assignment #1**

Assigned: January 8, 2019

Due: January 17, 2019



---

This assignment consists of four questions that require a short answer and one that requires you to generate some Python code. You can enter your answers and your code directly in a Colaboratory notebook and upload the **shareable** link for the your notebook as your homework submission.



---

#1.

(12 points) This question focuses on the topic of data collection and feature extraction. Consider a scenario in which we want to identify the person who hand-wrote a particular message. Collect data for this machine learning scenario by writing the word "Cougs" ten times. Also ask a friend or classmate to write this word ten times. Capture and upload an electronic image with these signatures. Analyzing these twenty images, write descriptions of at least ten specific features (types of strokes, lines, dots) that would best discriminate your handwriting from that of your friend.



---

#2.

(9 points) Generate decision trees that would represent the following boolean functions:



*   not(A) and not(B)
*   (A and B) or C
*   A XOR B     (here "XOR" refers to exclusive or)


---

#3.

(6 points) Assume that you are given the set of labeled training examples that appear below, where each attribute has possible values *a*, *b*, or *c*, and the target *Output* has values + or -. What is the information gain for each attribute in this dataset?


F1 | F2 | F3 | Output
--- | --- | --- | ---
a | a | a | +
c | b | c | +
c | a | c | +
b | a | a | -
a | b | c | -
b | b | c | -


What feature would be chosen as the root of a decision tree?



---

#4.

(12 points) Consider the training set accuracy and test set accuracy curves plotted in the graph below as a function of the decision tree size (number of nodes in the decision tree). From the accuracy value we can compute error as (1.0 - accuracy).

![](https://drive.google.com/uc?id=1LVeFRdtI0cg9es6b4E12HO0AZSgn78fj)

Can you suggest a way to determine the amount of overfit that exists in the learned decision tree model based on these curves? Explain / justify your answer.

Based on the graph above what size decision tree would you choose to use and why?



---

#5.

(80 points) In this problem you are asked to become familiar with python-based decision tree code and make modifications to the code. The code is based on a structure defined at machinelearningmastery.com. *Note that all of the code you write needs to be entirely your own, not copied from another existing program or using existing libraries that perform the specified functionality.*

The first thing to note here is that the function used to determine the ideal attribute for splitting is gini index, rather than information gain as we discussed in class. Instead of using the entropy measure $-p^+ log_2 p^+ - p^- log_2 p^-$, we now use the gini measure $p^+(1-p^+) + p^-(1-p^-)$ . You can test out this function by adding these lines to the bottom of the code segment:

print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))

print(gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]))







In [0]:
# Calculate the Gini index for a subset of the dataset
def gini_index(groups, classes):
   # count all samples at split point
   num_instances = float(sum([len(group) for group in groups]))

   gini = 0.0 # sum weighted Gini index for each group
   for group in groups:
      size = float(len(group))
      if size == 0: # avoid divide by zero
         continue
      score = 0.0
      # score the group based on the score for each class
      for class_val in classes:
         p = [row[-1] for row in group].count(class_val) / size
         score += p * p
      # weight the group score by its relative size
      gini += (1.0 - score) * (size / num_instances)
   return gini

Next, we define functions that test alternative attributes for splitting a node of the tree into two children (this code defines a binary decision tree). The test_split function divides the training data into two groups, one for each child. The get_split function determines the best attribute to split by calling test_split then evaluating the resulting data subsets using gini index.

You may notice an important different between this decision tree and one we discussed in class. This tree has numeric features, rather than discrete features with symbolic names. This adds a new level of complexity to the idea of splitting the tree. Instead of creating a separate child for each discrete value of the feature, we split the entire numeric range into two partitions based on a threshold value (this is the row[index] value in the select_attribute function). Data points whose value for the selected attribute is < threshold are assigned to the left child of the split node, the remaining data points are assigned to the right child. Rather than test all possible threshold values (of which there are an infinite number) to determine which yields the best gini value, only the values that are found in the actual dataset are tested.



In [0]:
# Create child splits for a node or make a leaf node
def split(node, max_depth, depth):
   left, right = node['groups']
   del(node['groups'])
   # check for a no split
   if not left or not right:
      node['left'] = node['right'] = create_leaf(left + right)
      return
   # check for max depth
   if depth >= max_depth:
      node['left'], node['right'] = create_leaf(left), create_leaf(right)
      return
   node['left'] = select_attribute(left)
   split(node['left'], max_depth, depth+1)
   node['right'] = select_attribute(right)
   split(node['right'], max_depth, depth+1)


# split the dataset based on an attribute and attribute value
def test_split(index, value, dataset):
   left, right = list(), list()
   for row in dataset:
      if row[index] < value:
         left.append(row)
      else:
         right.append(row)
   return left, right


# Select the best split point for a dataset
def select_attribute(dataset):
   class_values = list(set(row[-1] for row in dataset))
   b_index, b_value, b_score, b_groups = 999, 999, 999, None
   for index in range(len(dataset[0])-1):
      for row in dataset:
         groups = test_split(index, row[index], dataset)
         gini = gini_index(groups, class_values)
         if gini < b_score:
            b_index, b_value, b_score, b_groups = index, row[index], gini, groups
   return {'index':b_index, 'value':b_value, 'groups':b_groups}

Because the dataset we will eventually use for this assignment is quite large, we add a max_depth parameter to the tree. As the tree is built, if the depth limit is reached the node is not split further.
Similarly, if the subset of data at the current node is homogeneous (all the same class value), there is no need for a split. In either of these cases, the current node is considered a leaf node. The create_leaf function determines the class value that will be returned for that leaf node (based on the majority class label for training data that was assigned to the leaf node).

In [0]:
# Create a leaf node class value
def create_leaf(group):
   outcomes = [row[-1] for row in group]
   return max(set(outcomes), key=outcomes.count)

As shown in the code below, this version of the program starts with a hard-coded dataset. This dataset contains two numeric attributes and one integer class value. The main function defines the dataset, calls a function to train the decision tree and print the resulting tree structure.  You can try running this and seeing what type of tree structure is learned from the example dataset. Try varying the value of max_depth and see how this affects the tree structure.

In [0]:
# Build a decision tree
def build_tree(train, max_depth):
   root = select_attribute(train)
   split(root, max_depth, 1)
   return root
  
  
# Print a decision tree
def print_tree(node, depth=0):
   if depth == 0:
      print 'Tree:'
   if isinstance(node, dict):
      print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
      print_tree(node['left'], depth+1)
      print_tree(node['right'], depth+1)
   else:
      print('%s[%s]' % ((depth*' ', node)))
      
      
if __name__ == "__main__":
   dataset = [[2.771244718,1.784783929,0], [1.728571309,1.169761413,0],
              [3.678319846,2.812813570,0], [3.961043357,2.619950320,0],
              [2.999208922,2.209014212,0], [7.497545867,3.162953546,1],
              [9.00220326, 3.339047188,1], [7.444542326,0.476683375,1],
              [10.12493903,3.234550982,1], [6.642287351,3.319983761,1]]
   tree = build_tree(dataset, 1)
   print_tree(tree)

value 2.771244718 groups ([[1.728571309, 1.169761413, 0]], [[2.771244718, 1.784783929, 0], [3.678319846, 2.81281357, 0], [3.961043357, 2.61995032, 0], [2.999208922, 2.209014212, 0], [7.497545867, 3.162953546, 1], [9.00220326, 3.339047188, 1], [7.444542326, 0.476683375, 1], [10.12493903, 3.234550982, 1], [6.642287351, 3.319983761, 1]])
value 1.728571309 groups ([], [[2.771244718, 1.784783929, 0], [1.728571309, 1.169761413, 0], [3.678319846, 2.81281357, 0], [3.961043357, 2.61995032, 0], [2.999208922, 2.209014212, 0], [7.497545867, 3.162953546, 1], [9.00220326, 3.339047188, 1], [7.444542326, 0.476683375, 1], [10.12493903, 3.234550982, 1], [6.642287351, 3.319983761, 1]])
value 3.678319846 groups ([[2.771244718, 1.784783929, 0], [1.728571309, 1.169761413, 0], [2.999208922, 2.209014212, 0]], [[3.678319846, 2.81281357, 0], [3.961043357, 2.61995032, 0], [7.497545867, 3.162953546, 1], [9.00220326, 3.339047188, 1], [7.444542326, 0.476683375, 1], [10.12493903, 3.234550982, 1], [6.642287351, 3.319

The procedure to learn a decision tree is now complete. To use the tree for prediction of a new data point, we define a function predict that feeds the data point (called "row") through the tree (rooted at "node"). The value of the leaf node that is reached is returned. You can try the prediction function out by adding these three lines to the end of the main function:

   for row in dataset:
      prediction = predict(tree, row)
      print('Predicted=%d, Ground truth=%d' % (prediction, row[-1]))

The predicted values will likely match the ground truth values. This is not surprising since data points are being tested that were also used to train the tree.

In [0]:
# Make a prediction with a decision tree
def predict(node, row):
   if row[node['index']] < node['value']:
      if isinstance(node['left'], dict):
         return predict(node['left'], row)
      else:
         return node['left']
   else:
      if isinstance(node['right'], dict):
         return predict(node['right'], row)
      else:
         return node['right']

        
if __name__ == "__main__":
   dataset = [[2.771244718,1.784783929,0], [1.728571309,1.169761413,0],
              [3.678319846,2.812813570,0], [3.961043357,2.619950320,0],
              [2.999208922,2.209014212,0], [7.497545867,3.162953546,1],
              [9.00220326, 3.339047188,1], [7.444542326,0.476683375,1],
              [10.12493903,3.234550982,1], [6.642287351,3.319983761,1]]
   tree = build_tree(dataset, 3)
   print_tree(tree)
   for row in dataset:
      prediction = predict(tree, row)
      print('Predicted=%d, Ground truth=%d' % (prediction, row[-1]))

NameError: ignored

**Your role:**

In this homework assignment you need to make three changes to the code that has been provided.

* First, replace the gini measure with the information gain measure we described in class.

* Second, introduce functions that take the dataset and split into a subset that is used for training and a subset that is used for testing. The training set should represent 2/3 of the original data and testing will be the remaining 1/3 of the data. You can be as creative as you want in splitting the data into training and testing subsets. Now modify the main function to build the tree on the training data and test the predicted values on the testing data.  Instead of reporting each ground truth label, print the ratio of correctly-labeled testing data points (the predicted label matches the ground truth label) to the total number of testing data points.

* Third, instead of using the hard coded dataset that is provided here, I would like you to train and test your tree on the human activity recognition dataset that we described in class. A description of the dataset is online at http://eecs.wsu.edu/~cook/ml/hw/har_readme.txt and a csv version of the data is available online at http://eecs.wsu.edu/~cook/ml/hw/har.csv. As we mentioned in class, each data point in this set represents features extracted from accelerometer and gyroscope data. The two class values are 0 (representing "walking") and 1 (representing "sitting"). Use the google library and upload_files function to upload the data and the numpy library to process the csv file, as shown in class.

* Lastly, answer these questions. What observations can you make about the reported performance? Is it better than random guess?